<a href="https://colab.research.google.com/github/MsPery/colabwork_1/blob/main/data_munging_stuff_PerisOdhiambo_Ass1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
arrest = pd.read_csv('/content/drive/MyDrive/dataset/Arrest_Data_from_2010_to_Present.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
arrest.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,4248313,02/24/2015,1310.0,20.0,Olympic,2022.0,37.0,M,H,5.0,Burglary,F,459PC,BURGLARY,5TH,WILTON,"(34.0653, -118.314)"
1,191811472,05/03/2019,1700.0,18.0,Southeast,1802.0,23.0,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)"
2,4254777,02/26/2015,2010.0,19.0,Mission,1985.0,22.0,M,H,6.0,Larceny,M,459.5PC,SHOPLIFTING,8300 VAN NUYS BL,NaN,"(34.2216, -118.4488)"
3,5614161,04/29/2019,1040.0,8.0,West LA,842.0,41.0,M,H,3.0,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NaN,"(34.0508, -118.4592)"
4,5615197,04/30/2019,615.0,6.0,Hollywood,663.0,27.0,M,O,5.0,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)"


***bookings of arrestees that were made in 2018***

In [ ]:
arrest['Arrest Date'] = pd.to_datetime(arrest['Arrest Date'])

In [ ]:
arrests_2018 = arrest[arrest['Arrest Date'].dt.year == 2018]

In [ ]:
arrest_bookings_2018 = arrests_2018['Report ID'].nunique()

In [ ]:
print("The number of bookings of arrestees made in 2018 was:", arrest_bookings_2018)

The number of bookings of arrestees made in 2018 was: 42381


***bookings of arrestees that were made in each area and the area that has the most arrests in 2018***

In [ ]:
arrests_by_area = arrests_2018['Area Name'].value_counts()

In [ ]:
max_arrests_area = arrests_by_area.idxmax()

In [ ]:
print("Number of bookings made in each area in 2018:\n", arrests_by_area)
print("\nThe area with the most arrests in 2018 was:", max_arrests_area)

Number of bookings made in each area in 2018:
 Central        5621
Hollywood      5378
Rampart        4243
Pacific        3545
Van Nuys       2332
N Hollywood    2052
Southwest      1768
Mission        1701
Newton         1638
Harbor         1628
Northeast      1579
77th Street    1457
West LA        1415
Olympic        1350
Devonshire     1336
Topanga        1208
Hollenbeck      995
West Valley     955
Foothill        801
Southeast       764
Wilshire        615
Name: Area Name, dtype: int64

The area with the most arrests in 2018 was: Central


***average age of arrestees per each charge group***

In [ ]:
arrests_2018 = arrests_2018[~arrests_2018['Charge Group Description'].isin
                            (['Pre-Delinquency', 'Non-Criminal Detention'])]

In [ ]:
arrests_2018 = arrests_2018[arrests_2018['Charge Group Description'].notna()]

In [ ]:
avg_age_by_charge_group = arrests_2018.groupby('Charge Group Description')['Age'].mean()

In [ ]:
print("Average age of arrestees per charge group in 2018:")
print(avg_age_by_charge_group)

Average age of arrestees per charge group in 2018:
Charge Group Description
Against Family/Child              36.950355
Aggravated Assault                33.907359
Burglary                          30.366755
Disorderly Conduct                41.924290
Disturbing the Peace              39.727273
Driving Under Influence           34.428291
Drunkeness                        44.219972
Federal Offenses                  34.600000
Forgery/Counterfeit               35.066667
Fraud/Embezzlement                35.689076
Gambling                          40.557047
Homicide                          30.896552
Larceny                           32.725796
Liquor Laws                       42.540955
Miscellaneous Other Violations    36.252076
Miscellaneous Other Violatio{     17.000000
Moving Traffic Violations         33.085622
Narcotic Drug Laws                36.097954
Other Assaults                    33.451139
Prostitution/Allied               31.620470
Rape                              36.203704


***95% quantile of the age of the arrestee in 2018 for the considered charge groups***

In [ ]:
considered_charge_groups = ['Vehicle Theft', 'Robbery', 'Burglary', 'Receive Stolen Property']
arrests_2018 = arrests_2018[arrests_2018['Charge Group Description'].isin(considered_charge_groups)]


In [ ]:
quantile_95th = arrests_2018['Age'].quantile(0.95)

In [ ]:
print("95% quantile of the age of the arrestee in 2018 for the considered charge groups:")
print("is:", quantile_95th)

95% quantile of the age of the arrestee in 2018 for the considered charge groups:
is: 52.0


***arrest incidents occurred within 2 km from the Bradbury Building in 2018***

In [ ]:
import numpy as np


In [ ]:
arrests_2018 = arrests_2018[~((arrests_2018['Location'].str.startswith('0')) & 
                              (arrests_2018['Location'].str.endswith('0'))) ]

In [ ]:
lat1, lon1 = 34.050536, -118.247861

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the Earth in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [ ]:
arrests_2018[['Latitude', 'Longitude']] = arrests_2018['Location'].str.split(',', expand=True)


In [ ]:
arrests_2018['Latitude'] = pd.to_numeric(arrests_2018['Latitude'], errors='coerce')
arrests_2018['Longitude'] = pd.to_numeric(arrests_2018['Longitude'], errors='coerce')

In [ ]:
arrests_2018['Distance'] = arrests_2018.apply(lambda row: haversine(lat1, lon1, row['Latitude'], row['Longitude']) 
if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']) 
else np.nan, axis=1)

In [ ]:
arrests_within_2km = (arrests_2018['Distance'] <= 2).sum()

In [ ]:
print("Number of arrest incidents that occurred within 2 km from the Bradbury Building in 2018:", arrests_within_2km)

Number of arrest incidents that occurred within 2 km from the Bradbury Building in 2018: 0


***number of crime arrests in 2019***

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
arrests = arrest[arrest['Arrest Date'].dt.year.between(2010, 2018)]

In [ ]:
arrests_by_year = arrests.groupby('Arrest Date').size().reset_index(name='Num Arrests')

In [ ]:
model = LinearRegression()
model.fit(arrests_by_year[['Arrest Date']], arrests_by_year['Num Arrests'])

LinearRegression()

In [ ]:
arrests_2019 = round(model.predict([[2019]])[0])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:
print(f"Predicted number of crime arrests in 2019: {arrests_2019}")

Predicted number of crime arrests in 2019: 1895
